Component Library
==============
Hello! Welcome to the content library. Here you will find widgets that you can use in your own notebooks. You can either just copy paste them or call them like this: [Examples](https://nebaridia.equinor.com/user/isax/lab/tree/shared/analyst/Content/Examples.ipynb) # TODO add open in new tab icon

For info: 
> All dropdowns till the suffixed with "_dd" to signal that they are a dropdown. Other than that, the name is chosen to give them as much semantic meaning as possible

> First section describes getting dropdown irrespective of time, whereas the #TODOnAME describes getting dropdowns based time

> Copy-paste away!
 

In [ ]:
#| default_exp component-library

In [ ]:
#| export
import panel as pn
import panel_material_ui as pmui
from datetime import date, timedelta
import nebari_dia_adx.gremlin_connection as gc
import nebari_dia_adx.adx_connection as nbx
pn.extension()

# Dropdown menus

## Dropdown Menus - Basic
This section will describe how to get dropdowns for date, rigs, wellbores and sensors irrespective of time. 
If you want to use any of these, you can either copy-paste the code to where you want to use it, or you can call them.

If you want to to use drowdowns that are based on time, e.g. get rigs that were active on a spefici date, then check out the section below:  # TODO add ref to seciton belwo

### Date

In [ ]:
#| export
all_dates_dd = pmui.DatetimePicker()

# let's call the dropdown in order to visualize it:
all_dates_dd

DatetimePicker(design=<class 'panel_material_ui...., format='YYYY-MM-DD HH:mm:ss')

### Rigs

In [ ]:
#| export
rigs_query = f"""g.V().hasLabel('rigs')
                    .group()
                    .by(values('rigName'))
                    .by(values('id'))
                .unfold()  
                .order()
                    .by(select(keys).unfold())  
                .group()
                    .by(select(keys))  
                    .by(select(values))"""

rigs_options = gc.submit_query(rigs_query)[0]

all_rigs_dd = pmui.Select(options=rigs_options, searchable=True)

#  let's call the dropdown in order to visualize it:
pn.Row(all_rigs_dd, height=300)

Getting result took 0.703859806060791 s


Row(design=<class 'panel_material_ui...., height=300)
    [0] Select(design=<class 'panel_material_ui...., options={'48 FATH': 'rig_25439773'...}, searchable=True, value='rig_25439773')

### Wellbores

In [ ]:
#| export
# wellbores_query = f"""g.V().hasLabel('wellbores').valueMap('wellboreClassification', 'wellboreName', 'wellboreId', 'wellboreType','wellboreStatus' )"""
wellbores_query = f"""g.V().hasLabel('wellbores')
                    .group()
                    .by(values('wellboreName'))
                    .by(values('id'))
                .unfold()  
                .order()
                    .by(select(keys).unfold())  
                .group()
                    .by(select(keys))  
                    .by(select(values))"""
# wellbores_query = f"""g.V().hasLabel('wellbores')
                
#                     .order()
#                         .by(values('wellboreName'))  
#                      .group()
#                          .by(values('wellboreName'))
#                          .by(values('id'))
#                     """
wellbore_options = gc.submit_query(wellbores_query)[0]
# wellbore_options

Getting result took 0.9892330169677734 s


In [ ]:
#| export
all_wellbores_dd = pmui.Select(options=wellbore_options, searchable=True)

#  let's call the dropdown in order to visualize it:
pn.Row(all_wellbores_dd, height=300)

Row(design=<class 'panel_material_ui...., height=300)
    [0] Select(design=<class 'panel_material_ui...., options={'205/01-2': 'wellbore_189...}, searchable=True, value='wellbore_189151357')

### Sensors

In [ ]:
#| export
df_sensors = nbx.perform_query(f'''
TimeSeriesColumnsView ''', database= "Sensor Timeseries", table = 'TimeSeriesColumnsView' )

all_sensors_dd = pmui.Select(name='Sensors', options=dict(zip(df_sensors['FinalColumnValue'], df_sensors['ColumnName'])), searchable=True )


#  let's call the dropdown in order to visualize it:
pn.Row(all_sensors_dd, height=300)

Row(design=<class 'panel_material_ui...., height=300)
    [0] Select(design=<class 'panel_material_ui...., label='Sensors', name='Sensors', options={'APIT Surf Press (bar)': ...}, searchable=True, value='APIT Surf Press_bar_')

## Dropdown Menus - Time Based
This section describes getting dropdowns based on time. 
Copy-paste away!

The core idea here is that you first specific a time period, and then the dropdown will contain items that have operations on that date or during that time period.

### Date
Here we are getting a datepicker where the default date is yesterday's date. 

In [ ]:
#| export
dates_dd = pmui.DatetimePicker(value = date.today() - timedelta(days=1)) 

#  let's call the dropdown in order to visualize it:
pn.Row(dates_dd, height=300)

Row(design=<class 'panel_material_ui...., height=300)
    [0] DatetimePicker(design=<class 'panel_material_ui...., format='YYYY-MM-DD HH:mm:ss', value=datetime.datetime(2025, ...)

### Rigs

In [ ]:
#| export
def get_rigs(chosenDate = date.today() - timedelta(days=1) , end_date = None):
    end_date = (chosenDate + timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ") if end_date is None else end_date
    print(end_date)
    rigs_query = f"""
        g.V().hasLabel('operations')
            .has('startTimeUTC', gt('{chosenDate}')).has('endTimeUTC', lt('{end_date}'))
            .in('PERFORMED')
            .group()
                .by(values('properties', 'rigName'))
                .by(values('id'))
            .unfold()  
            .order()
                .by(select(keys).unfold())  
            .group()
                .by(select(keys))  
                .by(select(values))
        """
    return gc.submit_query(rigs_query)[0]



rigs_dd = pmui.Select(options=pn.bind(gc.get_rigs, dates_dd.param.value), searchable=True)

#  let's call the dropdown in order to visualize it:
pn.Row(rigs_dd, height=300)

Getting result took 0.32072877883911133 s


Row(design=<class 'panel_material_ui...., height=300)
    [0] Select(design=<class 'panel_material_ui...., searchable=True)

In [ ]:
# rigs_dd.param.objects #0x7fcbc4348720 0x7fcbc42ccea0  0x7fcbc6e3da00 0x7fcbb9c03710

### Wellbores

In [ ]:
#| export
def get_wellbores(rig, chosenDate = date.today() - timedelta(days=1) , end_date = None):
    end_date = (chosenDate + timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ") if end_date is None else end_date
    wellbores_query =  f"""g.V('{rig}')
                    .out('PERFORMED').has('startTimeUTC', gt('{dates_dd.value}')).has('endTimeUTC',lt('{end_date}'))
                    .out('ON')
                       .group().by(values('properties', 'wellboreName')).by(values('id'))
                       .unfold()  
                        .order()
                            .by(select(keys).unfold())  
                        .group()
                            .by(select(keys))  
                            .by(select(values))
                    """
    return gc.submit_query(wellbores_query)[0]



wellbore_dd = pmui.Select(options=pn.bind(get_wellbores, rigs_dd.param.value, dates_dd.param.value), searchable=True)

#  let's call the dropdown in order to visualize it:
pn.Row(wellbore_dd, height=300)

Getting result took 0.3737502098083496 s


Row(design=<class 'panel_material_ui...., height=300)
    [0] Select(design=<class 'panel_material_ui...., searchable=True)

### Sensors

In [ ]:
#| export
df_sensors = nbx.perform_query(f'''
TimeSeriesColumnsView ''', database= "Sensor Timeseries", table = 'TimeSeriesColumnsView' )

all_sensors_dd = pmui.Select(name='Sensors', options=dict(zip(df_sensors['FinalColumnValue'], df_sensors['ColumnName'])), searchable=True )


#  let's call the dropdown in order to visualize it:
pn.Row(all_sensors_dd, height=300)

Row(design=<class 'panel_material_ui...., height=300)
    [0] Select(design=<class 'panel_material_ui...., label='Sensors', name='Sensors', options={'APIT Surf Press (bar)': ...}, searchable=True, value='APIT Surf Press_bar_')